In [1]:
import plotly.express as px
import pandas as pd
from config import StocksConfig as config
import psycopg2

conn = psycopg2.connect(database=config.DB_NAME, host=config.DB_HOST, user=config.DB_USER, password=config.DB_PASS, port=config.DB_PORT)

In [14]:
#  How <ticker> price changed over time?
query = """
SELECT symbol, time_bucket('12 hours', stock_datetime) AS time_frame, last(price_close, stock_datetime) AS last_closing_price
FROM stocks_new
WHERE symbol = 'AAPL'
GROUP BY time_frame, symbol
ORDER BY time_frame
"""

df = pd.read_sql(query, conn)

figure = px.line(df, x='time_frame', y='last_closing_price')
figure.show()

In [3]:
    # How <ticker> trading volume changed over time?
    query = """
        SELECT time_bucket('7 days', stock_datetime) AS time_frame, sum(trading_volume) AS volume
        FROM stocks_new
        WHERE symbol = 'AAPL'
        GROUP BY time_frame
        ORDER BY time_frame
    """
    df = pd.read_sql(query, conn)

    fig = px.line(df, x='time_frame', y='volume')
    fig.show()

In [11]:
    # Which tickers had the most transaction volume in the past 14 days?
    query = """
        SELECT t.name, sum(trading_volume) AS volume
        FROM stocks_new s
        INNER JOIN ticker t ON t.symbol = s.symbol
        WHERE (now() - date(stock_datetime)) < INTERVAL '14 day'
        GROUP BY s.symbol, t.name
        ORDER BY volume DESC
        LIMIT 5
    """
    df = pd.read_sql(query, conn)

    fig = px.bar(df, x='name', y='volume')
    fig.show()

In [12]:
# FAANG prices over time
    query = """
    SELECT symbol, time_bucket('30 days', stock_datetime) AS time_frame, last(price_close, stock_datetime) AS last_closing_price
    FROM stocks_new
    WHERE symbol in ('AAPL', 'FB', 'AMZN', 'NFLX', 'GOOG')
    GROUP BY time_frame, symbol
    ORDER BY time_frame
    """
    df = pd.read_sql(query, conn)

    fig = px.line(df, x='time_frame', y='last_closing_price', color='symbol')
    fig.show()

In [13]:
    # Which symbols had the biggest daily gain/loss?
    query = """
    SELECT symbol, time_frame, max((closing_price-opening_price)/closing_price*100) AS price_change_pct
    FROM ( 
        SELECT 
        symbol, 
        time_bucket('1 day', stock_datetime) AS time_frame, 
        first(price_open, stock_datetime) AS opening_price, 
        last(price_close, stock_datetime) AS closing_price
        FROM stocks_new
        GROUP BY time_frame, symbol
    ) s
    GROUP BY symbol, s.time_frame
    ORDER BY price_change_pct DESC
    LIMIT 5
    """
    df = pd.read_sql(query, conn)
    print(df)

  symbol time_frame  price_change_pct
0     ZM 2021-06-07         19.326112
1   TSLA 2021-03-09         15.696928
2   SNAP 2021-02-05         14.606391
3     ZM 2021-03-01         13.961712
4    NKE 2021-06-24         12.380328


In [1]:
import plotly.express as px
import pandas as pd
from config import StocksConfig as config
import psycopg2

conn = psycopg2.connect(database=config.DB_NAME, host=config.DB_HOST, user=config.DB_USER, password=config.DB_PASS, port=config.DB_PORT)

In [2]:
query = """
   SELECT symbol, time_frame, max((closing_price-opening_price)/closing_price) AS price_change_pct
    FROM ( 
        SELECT 
        symbol, 
        time_bucket('7 day', stock_datetime) AS time_frame, 
        first(price_open, stock_datetime) AS opening_price, 
        last(price_close, stock_datetime) AS closing_price
        FROM stocks_new
        WHERE symbol IN ('AAPL', 'FB', 'GOOG')
        GROUP BY time_frame, symbol
    ) s
    GROUP BY symbol, s.time_frame
    ORDER BY time_frame
"""

df = pd.read_sql(query, conn)
figure = px.line(df, x="time_frame", y="price_change_pct", color="symbol", title="Apple, Facebook, Google comparison")
figure = figure.update_layout(yaxis={'tickformat': '.2%'})
figure.show()

In [8]:
query = """
   SELECT symbol, time_frame, max((closing_price-opening_price)/closing_price) AS price_change_pct
    FROM ( 
        SELECT 
        symbol, 
        time_bucket('1 day', stock_datetime) AS time_frame, 
        first(price_open, stock_datetime) AS opening_price, 
        last(price_close, stock_datetime) AS closing_price
        FROM stocks_new
        WHERE symbol IN ('ZM', 'AMZN')
        GROUP BY time_frame, symbol
    ) s
    GROUP BY symbol, s.time_frame
    ORDER BY time_frame
"""
df = pd.read_sql(query, conn)
figure = px.scatter(df, x="price_change_pct", color="symbol", title="Distribution of daily price changes (Zoom,  Amazon)")
figure = figure.update_layout(xaxis={'tickformat': '.2%'})
figure.show()

In [39]:
import plotly.graph_objects as go

In [50]:
query = """
    SELECT time_bucket('15 min', stock_datetime) AS time_frame, 
    FIRST(price_open, stock_datetime) AS price_open, 
    LAST(price_close, stock_datetime) AS price_close,
    MAX(price_high) AS price_high,
    MIN(price_low) AS price_low
    FROM stocks_new2
    WHERE symbol = 'ZM' AND date(stock_datetime) = date('2021-06-15') 
    GROUP BY time_frame
"""
df = pd.read_sql(query, conn)
figure = go.Figure(data=[go.Candlestick(x=df['time_frame'],
                open=df['price_open'],
                high=df['price_high'],
                low=df['price_low'],
                close=df['price_close'])])
figure.show()

In [7]:
query = """
    SELECT time_bucket('{time_frame}', stock_datetime) AS time_frame,
    last(price_close, stock_datetime) AS last_closing_price
    FROM stocks_new
    WHERE symbol = '{symbol}'
    GROUP BY time_frame
    ORDER BY time_frame
""".format(time_frame="1 day", symbol="AAPL")
df = pd.read_sql(query, conn)
fig = px.line(df, x='time_frame', y='last_closing_price', title="Apple's stock price over time")
fig.show()

In [2]:
query = """
    SELECT time_bucket('1 days', stock_datetime) AS time_frame, sum(trading_volume) AS volume
    FROM stocks_new
    WHERE symbol = 'AAPL'
    GROUP BY time_frame
    ORDER BY time_frame
""".format(time_frame="7 days", symbol="AAPL")
df = pd.read_sql(query, conn)
fig = px.line(df, x='time_frame', y='volume', title="Apple's trading volume over time")
fig.show()

In [14]:
query = """
    SELECT symbol, sum(trading_volume) AS volume
    FROM stocks_new
    WHERE (now() - date(stock_datetime)) < INTERVAL '{time_frame}'
    GROUP BY symbol
    ORDER BY volume DESC
    LIMIT 5
""".format(time_frame="14 day")
df = pd.read_sql(query, conn)
fig = px.bar(df, x='symbol', y='volume', title="Most traded symbols in the last 14 days")
fig.show()

In [16]:
query = """
    SELECT symbol, time_bucket('{time_frame}', stock_datetime) AS time_frame, 
    last(price_close, stock_datetime) AS last_closing_price
    FROM stocks_new
    WHERE symbol in {symbol}
    GROUP BY time_frame, symbol
    ORDER BY time_frame
""".format(time_frame="7 days", symbol="('AAPL', 'FB', 'AMZN', 'NFLX', 'GOOG')")
df = pd.read_sql(query, conn)
fig = px.line(df, x='time_frame', y='last_closing_price', color='symbol', title="FAANG prices over time")
fig.show()

In [19]:
query = """
SELECT symbol, time_frame, max((closing_price-opening_price)/closing_price*100) AS price_change_pct
FROM ( 
    SELECT 
    symbol, 
    time_bucket('{time_frame}', stock_datetime) AS time_frame, 
    first(price_open, stock_datetime) AS opening_price, 
    last(price_close, stock_datetime) AS closing_price
    FROM stocks_new
    GROUP BY time_frame, symbol
) s
GROUP BY symbol, s.time_frame
ORDER BY price_change_pct {orderby}
LIMIT 5
""".format(time_frame="7 days", orderby="DESC")
df = pd.read_sql(query, conn)
print(df)

  symbol time_frame  price_change_pct
0     ZM 2021-06-07         24.586495
1   TSLA 2021-01-04         18.280314
2     BA 2021-03-08         17.745225
3   SNAP 2021-02-01         16.149649
4   TSLA 2021-03-08         15.842941


In [22]:
query = """
   SELECT symbol, time_frame, max((closing_price-opening_price)/closing_price) AS price_change_pct
    FROM ( 
        SELECT 
        symbol, 
        time_bucket('7 day', stock_datetime) AS time_frame, 
        first(price_open, stock_datetime) AS opening_price, 
        last(price_close, stock_datetime) AS closing_price
        FROM stocks_new
        WHERE symbol IN ('AAPL', 'FB', 'GOOG')
        GROUP BY time_frame, symbol
    ) s
    GROUP BY symbol, s.time_frame
    ORDER BY time_frame
"""

df = pd.read_sql(query, conn)
figure = px.line(df, x="time_frame", y="price_change_pct", color="symbol", title="Apple, Facebook, Google weekly price changes")
figure = figure.update_layout(yaxis={'tickformat': '.2%'})
figure.show()

In [24]:
query = """
   SELECT symbol, time_frame, max((closing_price-opening_price)/closing_price) AS price_change_pct
    FROM ( 
        SELECT 
        symbol, 
        time_bucket('{time_frame}', stock_datetime) AS time_frame, 
        first(price_open, stock_datetime) AS opening_price, 
        last(price_close, stock_datetime) AS closing_price
        FROM stocks_new
        WHERE symbol IN {symbols}
        GROUP BY time_frame, symbol
    ) s
    GROUP BY symbol, s.time_frame
    ORDER BY time_frame
""".format(time_frame="7 days", symbols="('AAPL', 'FB', 'GOOG')")
df = pd.read_sql(query, conn)
figure = px.line(df, x="time_frame", y="price_change_pct", color="symbol", title="Apple, Facebook, Google weekly price changes")
figure = figure.update_layout(yaxis={'tickformat': '.2%'})
figure.show()

In [38]:
import plotly.graph_objects as go
query = """
    SELECT time_bucket('{time_frame}', stock_datetime) AS time_frame, 
    FIRST(price_open, stock_datetime) AS price_open, 
    LAST(price_close, stock_datetime) AS price_close,
    MAX(price_high) AS price_high,
    MIN(price_low) AS price_low
    FROM stocks_new2
    WHERE symbol = '{symbol}' AND date(stock_datetime) = date('{date}') 
    GROUP BY time_frame
""".format(time_frame="15 min", symbol="AAPL", date="2021-06-09")
df = pd.read_sql(query, conn)
figure = go.Figure(data=[go.Candlestick(x=df['time_frame'],
                   open=df['price_open'],
                   high=df['price_high'],
                   low=df['price_low'],
                   close=df['price_close'],)])
figure.update_layout(title="15-min candlestick chart of Apple, 2021-06-09")
figure.show()

In [40]:
query = """
   SELECT symbol, time_frame, max((closing_price-opening_price)/closing_price) AS price_change_pct
    FROM ( 
        SELECT 
        symbol, 
        time_bucket('1 day', stock_datetime) AS time_frame, 
        first(price_open, stock_datetime) AS opening_price, 
        last(price_close, stock_datetime) AS closing_price
        FROM stocks_new
        WHERE symbol IN ('ZM', 'AMZN')
        GROUP BY time_frame, symbol
    ) s
    GROUP BY symbol, s.time_frame
    ORDER BY time_frame
"""
df = pd.read_sql(query, conn)
figure = px.scatter(df, x="price_change_pct", color="symbol", title="Distribution of daily price changes (Amazon, Zoom)")
figure = figure.update_layout(xaxis={'tickformat': '.2%'})
figure.show()

In [41]:
query = """
   SELECT symbol, time_frame, max((closing_price-opening_price)/closing_price) AS price_change_pct
    FROM ( 
        SELECT 
        symbol, 
        time_bucket('{time_frame}', stock_datetime) AS time_frame, 
        first(price_open, stock_datetime) AS opening_price, 
        last(price_close, stock_datetime) AS closing_price
        FROM stocks_new
        WHERE symbol IN {symbols}
        GROUP BY time_frame, symbol
    ) s
    GROUP BY symbol, s.time_frame
    ORDER BY time_frame
""".format(time_frame="1 day", symbols="('ZM', 'AMZN')")
df = pd.read_sql(query, conn)
figure = px.scatter(df, x="price_change_pct", color="symbol", title="Distribution of daily price changes (Amazon, Zoom)")
figure = figure.update_layout(xaxis={'tickformat': '.2%'})
figure.show()

In [6]:
query = """
    SELECT time_bucket('{time_frame}', stock_datetime) AS time_frame, sum(trading_volume) AS volume
    FROM stocks_new
    WHERE symbol = '{symbol}'
    GROUP BY time_frame
    ORDER BY time_frame
""".format(time_frame="1 day", symbol="AAPL")
df = pd.read_sql(query, conn)
fig = px.line(df, x='time_frame', y='volume', title="Apple's daily trading volume over time")
fig.show()